## Crawling & Crawing

### 1. 실전 예제: 크롤링
* 이커머스(지마켓) 베스트100 상품 타이틀/가격 추출하기

In [ ]:
# 여기는 내가 작성

In [1]:
import requests
import openpyxl
from bs4 import BeautifulSoup

# 엑셀 파일에 쓰기 ( 랭킹, 상품명, 판매가격, 제품 상세 링크, 판매 업체 )
excel_file = openpyxl.Workbook()
excel_sheet = excel_file.active
excel_sheet.append(['랭킹', '상품명', '판매가격', '제품 상세 링크', '판매 업체'])
excel_sheet.column_dimensions['B'].width = 80
excel_sheet.column_dimensions['C'].width = 20
excel_sheet.column_dimensions['D'].width = 80
excel_sheet.column_dimensions['E'].width = 20


# 크롤링
URL = "http://corners.gmarket.co.kr/Bestsellers?viewType=G&groupCode=G06"

res = requests.get(URL)
if res.status_code == 200:
    soup = BeautifulSoup(res.content, 'html.parser')
    cards = soup.select('.best-list')[1].select('ul > li')

    for index, card in enumerate(cards):
        title = card.select_one('.itemname')    # 상품명
        price = card.select_one('.s-price > strong')    # 판매 가격
        link = title['href']    # 제품 상세 링크

        res_info = requests.get(link)
        if res_info.status_code == 200:
            print(f'Running: {index + 1}')
            soup_info = BeautifulSoup(res_info.content, 'html.parser')
            provider_info = soup_info.select_one('.text__seller > a')   # 판매 업체

            excel_row = [index + 1, title.get_text(), price.get_text(),
                         link, provider_info.get_text()]
            excel_sheet.append(excel_row)
            # 엑셀 파일 D열에 링크를 하이퍼링크로 만든다
            excel_sheet.cell(row=index+2, column=4).hyperlink = link


cells = ['A1', 'B1', 'C1', 'D1', 'E1']
for cell_name in cells:
    cell = excel_sheet[cell_name]
    cell.alignment = openpyxl.styles.Alignment(horizontal='center')    # 가운데 정렬
    cell.font = openpyxl.styles.Font(color="01579B")    # 폰트 색깔 변경

excel_file.save("gmarket_best.xlsx")
excel_file.close()
print('Done!')


1 [삼성전자](사전예약판매) 갤럭시S21 5G 자급제폰 SM-G991N 999,900원 삼성공식온라인판매점
2 [삼성전자](사전예약판매) 갤럭시S21 5G 자급제폰 SM-G991N 999,900원 삼성공식온라인판매점
3 [블루스카이]인증점 삼성 공기청정기 AX60N5580WDD 전국무료배송 286,000원 삼성공식다원
4 [삼성전자](사전판매) 갤럭시S21 울트라 5G 자급제폰 SM-G998N 1,452,000원 삼성공식온라인판매점
5 [다이슨]본사  다이슨 에어랩 스타일러 컴플리트 545,090원 다이슨코리아유한회사
6 [삼성전자](사전판매) 갤럭시S21 울트라 5G 자급제폰 SM-G998N 1,452,000원 삼성공식온라인판매점
7 [삼성전자]갤럭시버즈 프로 블루투스이어폰 ANC SM-R190 바이올렛 239,800원 jeyoumall
8 [샤오미]샤오미 미밴드5 여우미 공식수입 정식 한글판(미개봉) 38,900원 스마일배송
9 [삼성전자](사전판매) 갤럭시S21 플러스 5G 자급제폰 SM-G996N 1,199,000원 삼성공식온라인판매점
10 [애플]Apple 정품 에어팟 프로 MWP22KH/A 257,530원 Apple공식판매점
11 SKT 기변/번이 아이폰12 MINI_128G 완납/선약 708,400원 피에스앤마케팅
12 SKT 기변/번이 아이폰 12 PRO 256G 완납/선택약정 1,164,460원 CJ오쇼핑
13 윤남텍 간편세척 초음파 다목적 가습기/YN-101/G마켓 69,000원 윤남텍
14 [닌텐도]닌텐도 모여봐요 동물의 숲(SWITCH) 5천원 E캐시 증정 61,060원 지피클럽
15 [에이엠]am 멀티 다지기 FC-J18 (용기 3개  보관커버 포함) + 49,900원 GS홈쇼핑
16 [삼성전자]갤럭시버즈 프로 블루투스이어폰 ANC SM-R190 블랙 239,800원 jeyoumall
17 무선 블루투스 5.0 이어폰 3개 1+1+1 LED 27,400원 cylin678
18 [삼성전자]혜택56만원대)제트2.0 VS15R8577S5CW

In [ ]:
# 여기는 모범 답안

In [ ]:
import requests
from bs4 import BeautifulSoup

res = requests.get('http://corners.gmarket.co.kr/Bestsellers?viewType=G&groupCode=G06')
soup = BeautifulSoup(res.content, 'html.parser')
bestlists = soup.select('div.best-list')
bestitems = bestlists[1]
products = bestitems.select('ul > li')

for index, product in enumerate(products):
    title = product.select_one('a.itemname')
    price = product.select_one('div.s-price > strong')
    print (title.get_text(), price.get_text(), title['href'])

### 2. 실전 예제: 크롤링 & 크롤링
* 이커머스(지마켓) 베스트100 상품 타이틀/가격, 그리고 판매업체 추출하기

In [ ]:
import requests
from bs4 import BeautifulSoup

res = requests.get('http://corners.gmarket.co.kr/Bestsellers?viewType=G&groupCode=G06')
soup = BeautifulSoup(res.content, 'html.parser')
bestlists = soup.select('div.best-list')
bestitems = bestlists[1]
products = bestitems.select('ul > li')

for index, product in enumerate(products):
    title = product.select_one('a.itemname')
    price = product.select_one('div.s-price > strong')
    res_info = requests.get(title['href'])
    soup_info = BeautifulSoup(res_info.content, 'html.parser')
    provider_info = soup_info.select_one('div.item-topinfo > div.item-topinfo_headline > p > a > strong')
    print (title.get_text(), price.get_text(), title['href'], provider_info.get_text())

### 3. 실전 예제: 크롤링 & 크롤링 + 엑셀
* 이커머스(지마켓) 베스트100 상품 타이틀/가격, 그리고 판매업체 추출하기

In [ ]:
import requests, openpyxl
from bs4 import BeautifulSoup

excel_file = openpyxl.Workbook()
excel_sheet = excel_file.active
excel_sheet.append(['랭킹', '상품명', '판매가격', '상품상세링크', '판매업체'])
excel_sheet.column_dimensions['B'].width = 80
excel_sheet.column_dimensions['C'].width = 20
excel_sheet.column_dimensions['D'].width = 80
excel_sheet.column_dimensions['E'].width = 20

res = requests.get('http://corners.gmarket.co.kr/Bestsellers?viewType=G&groupCode=G06')
soup = BeautifulSoup(res.content, 'html.parser')

bestlists = soup.select('div.best-list')
bestitems = bestlists[1]
products = bestitems.select('ul > li')

for index, product in enumerate(products):
    title = product.select_one('a.itemname')
    price = product.select_one('div.s-price > strong')
    
    res_info = requests.get(title['href'])
    soup_info = BeautifulSoup(res_info.content, 'html.parser')
    provider_info = soup_info.select_one('div.item-topinfo > div.item-topinfo_headline > p > a > strong')
    print(index + 1, title.get_text(), price.get_text(), title['href'], provider_info.get_text())
    excel_sheet.append([index + 1, title.get_text(), price.get_text(), title['href'], provider_info.get_text()])
    excel_sheet.cell(row=index+2 , column=4).hyperlink = title['href']


cell_A1 = excel_sheet['A1'] # 셀 선택하기
cell_A1.alignment = openpyxl.styles.Alignment(horizontal='center') # 중앙정렬하기
cell_A1.font = openpyxl.styles.Font(color="01579B") # 폰트 색깔 바꾸기
# 색상값 찾기: https://material.io/design/color/#tools-for-picking-colors

cell_B1 = excel_sheet['B1'] # 셀 선택하기
cell_B1.alignment = openpyxl.styles.Alignment(horizontal='center') # 중앙정렬하기
cell_B1.font = openpyxl.styles.Font(color="01579B") # 폰트 색깔 바꾸기
# 색상값 찾기: https://material.io/design/color/#tools-for-picking-colors

cell_C1 = excel_sheet['C1'] # 셀 선택하기
cell_C1.alignment = openpyxl.styles.Alignment(horizontal='center') # 중앙정렬하기
cell_C1.font = openpyxl.styles.Font(color="01579B") # 폰트 색깔 바꾸기
# 색상값 찾기: https://material.io/design/color/#tools-for-picking-colors

cell_D1 = excel_sheet['D1'] # 셀 선택하기
cell_D1.alignment = openpyxl.styles.Alignment(horizontal='center') # 중앙정렬하기
cell_D1.font = openpyxl.styles.Font(color="01579B") # 폰트 색깔 바꾸기
# 색상값 찾기: https://material.io/design/color/#tools-for-picking-colors

cell_E1 = excel_sheet['E1'] # 셀 선택하기
cell_E1.alignment = openpyxl.styles.Alignment(horizontal='center') # 중앙정렬하기
cell_E1.font = openpyxl.styles.Font(color="01579B") # 폰트 색깔 바꾸기
# 색상값 찾기: https://material.io/design/color/#tools-for-picking-colors

excel_file.save('BESTPRODUCT_COM_GMARKET.xlsx')
excel_file.close()